chat with Multiple doc(pdfs,docs,txt,pptx) using langchain and astraDB

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader

In [ ]:
loader=DirectoryLoader() # path of file where data is present 

In [ ]:
spliter=RecursiveCharacterTextSplitter(chunk_siz=512,chunk_overlap=64)

In [ ]:
docs=loader.load_and_split(text_splitter=spliter)

In [ ]:
len(docs)

In [ ]:
import os
from langchain_core.documents import Document
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import  GoogleGenerativeAIEmbeddings,ChatGoogleGenerativeAI

In [ ]:
from dotenv import load_dotenv
load_dotenv()
import google.generativeai as genai
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

In [ ]:
llm=ChatGoogleGenerativeAI(model="models/gemini-2.5-pro")

In [ ]:
from langchain_astradb import AstraDBVectorStore
from langchain.indexes import VectorstoreIndexCreator

In [ ]:
ASTRA_DB_API_ENDPOINT="ENDPOINT"
ASTRA_DB_APPLICATION_TOKEN="APPLICATION_TOKEN"
ASTRA_DB_KEYSPACE="default_keyspace"

In [ ]:
vstore=AstraDBVectorStore(
    embeddding=GoogleGenerativeAIEmbeddings(),
    collection_name="multidoc_vector",
    api_endpoint=ASTRA_DB_API_ENDPOINT,
    token=ASTRA_DB_APPLICATION_TOKEN,
    namespace=ASTRA_DB_KEYSPACE,
    
)

In [ ]:
inserted_ids=vstore.add_documents(docs)
print(f"\nInserted{len(inserted_ids)} documents.")

In [ ]:
retriever=vstore.as_retriever(search_kwargs={"k":3})

In [ ]:
retriever

In [ ]:
prompt_template="""
You are an AI philosopher drawing inslights from the roadmap of "rag",llama3", and "genai."
craft thoughtful answers based on this roadmap,mixing and matching existing paths.
your responses should be concise and strictly related to the provided context.
ROADMAP CONTEXT:
{context}
QUESTION:{question}
YOUR ANSWER:
"""

In [ ]:
prompt_template=ChatPromptTemplate(prompt_template)

In [ ]:
chain=(
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

In [ ]:
chain.invoke("can you tell me the road of generative ai?")